클리앙 알뜰구매 게시판에 대해서 살펴보고, 댓글을 수정하고 정제하는 과정에 대해서 살펴본다.

https://www.clien.net/service/board/jirum

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from datetime import datetime

## 클리앙 알뜰 구매 게시판

In [2]:
url = "https://www.clien.net/service/board/jirum"
page = 1
params = {'po': page - 1}

res = requests.get(url, params=params)

In [3]:
res

<Response [200]>

In [4]:
html = res.text
soup = BeautifulSoup(html, "html.parser")

In [5]:
tag_list = soup.select(".contents_jirum .list_item")
len(tag_list)

30

In [6]:
tag = tag_list[0]
title_tag = tag.select_one("[data-role=list-title-text]")
title_url = urljoin(url, title_tag['href'])
title_text = title_tag.text

print(title_text, title_url)

[Amazon/직배] USB C Hub AUKEY 8-in-1 Type C Hub https://www.clien.net/service/board/jirum/14182612?od=T31&po=0&category=&groupCd=


In [7]:
user_id = tag["data-author-id"]
board_sn = tag["data-board-sn"]
comment_count = tag["data-comment-count"]
print(user_id, board_sn, comment_count)

jbk958 14182612 4


In [8]:
try:
    nickname = tag.select_one('.nickname img')['alt']
except TypeError:
    nickname = tag.select_one('.nickname').text.strip()
    
print(nickname)

넌생각이있냐없냐


In [9]:
# 투표 or 좋아요 ..
hit_number = tag.select_one('.hit').text

In [10]:
# 게시 시간을 추출하여, 문자열을 시간 객체로 변환
created_at_string = tag.select_one('.timestamp').text
created_at = datetime.strptime(created_at_string, "%Y-%m-%d %H:%M:%S")
created_at

datetime.datetime(2019, 10, 19, 21, 0, 16)

### 정보합치기

In [16]:
item_list = []

def get_nickname(tag):
    try:
        nickname = tag.select_one('.nickname img')['alt']
    except TypeError:
        nickname = tag.select_one('.nickname').text.strip()
    return nickname
    
def get_hit_num(tag):
    hit_number = tag.select_one('.hit').text
    if hit_number.endswith('k'):
        return float(hit_number[:-1]) * 1000
    else:
        return int(hit_number)
    
for tag in tag_list:
    is_deleted = "삭제" in tag.text
    if is_deleted != True:  # 삭제된 게시물 제외
        title_tag = tag.select_one("[data-role=list-title-text]")
        #created_at_string = tag.select_one('.timestamp').text
        item_list.append(dict(
            title_url = urljoin(url, title_tag['href']),
            title = title_tag.text,
            user_id = tag["data-author-id"],
            board_sn = tag["data-board-sn"],
            comment_count = int(tag["data-comment-count"]),
            nickname = get_nickname(tag),
            hit_number = get_hit_num(tag),
            created_at = datetime.strptime(tag.select_one('.timestamp').text, "%Y-%m-%d %H:%M:%S")
        ))

In [17]:
item_list

[{'title_url': 'https://www.clien.net/service/board/jirum/14182612?od=T31&po=0&category=&groupCd=',
  'title': '[Amazon/직배] USB C Hub AUKEY 8-in-1 Type C Hub',
  'user_id': 'jbk958',
  'board_sn': '14182612',
  'comment_count': 4,
  'nickname': '넌생각이있냐없냐',
  'hit_number': 4193,
  'created_at': datetime.datetime(2019, 10, 19, 21, 0, 16)},
 {'title_url': 'https://www.clien.net/service/board/jirum/14182442?od=T31&po=0&category=&groupCd=',
  'title': '현대카드 미국 Amazon 전상품 할인코드 (한국시간 21일 0시~)',
  'user_id': 'whdxoca',
  'board_sn': '14182442',
  'comment_count': 75,
  'nickname': '쥐즐',
  'hit_number': 13000.0,
  'created_at': datetime.datetime(2019, 10, 19, 20, 3, 42)},
 {'title_url': 'https://www.clien.net/service/board/jirum/14182013?od=T31&po=0&category=&groupCd=',
  'title': '[AmazonJP] 은련 8천엔이상 결제시 1500엔 할인',
  'user_id': 'mu4215',
  'board_sn': '14182013',
  'comment_count': 7,
  'nickname': '맑음7',
  'hit_number': 3361,
  'created_at': datetime.datetime(2019, 10, 19, 17, 48, 30)},
 {'ti

In [18]:
# 개별 게시물의 댓글 크롤링
post_url = "https://www.clien.net/service/board/jirum/14182612?od=T31&po=0&category=&groupCd="
res = requests.get(post_url)
res

<Response [200]>

In [24]:
html = res.text
soup = BeautifulSoup(html, "html.parser")
tag_list = soup.select_one('.comment')

### 댓글

In [29]:
comment_list_url = "https://www.clien.net/service/board/commentWriterList/"
user_id = 'jbk958'
url = urljoin(comment_list_url, user_id)
res = requests.get(url)
res

<Response [200]>

In [31]:
html = res.text
soup = BeautifulSoup(html, 'html.parser')
tag_list = soup.select('.comment')
len(tag_list)

8

In [36]:
tag = tag_list[0]
short_name = tag.select_one('.shortname').text
desc = tag.select('span')[1].text

'가시광선이 아니라 엑스선을 보는 외계인이 있다면...인간은 스켈레톤처럼 보이겠네요... 감마선을 보는 외계인이 있다면 인간은 투명인간일거고..'

In [37]:
comment_list = []
for tag in tag_list:
    short_name = tag.select_one('.shortname').text
    desc = tag.select('span')[1].text
    comment_list.append(
        {
            "short_name": short_name,
            "desc": desc
        }
    )

In [38]:
comment_list

[{'short_name': '모공',
  'desc': '가시광선이 아니라 엑스선을 보는 외계인이 있다면...인간은 스켈레톤처럼 보이겠네요... 감마선을 보는 외계인이 있다면 인간은 투명인간일거고..'},
 {'short_name': '아이폰', 'desc': '경험상은 프랑스가 좀 얇더라구요...왜 그런지는 모르겠습니다..'},
 {'short_name': '모공',
  'desc': '조국장관 증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증증조부쯤 되는 분은 곰한테 마늘이랑 쑥만 먹이면 사람된다고 거짓말 했다던데요'},
 {'short_name': '모공', 'desc': '닙스면 무조건가야죠....'},
 {'short_name': '모공', 'desc': '저도 줄서봅니다!'},
 {'short_name': '모공',
  'desc': '저는 다 읽어보지는 못했지만 나의한국근현대사, 역사의역사가 취향이고어떻게살것인가가 제일 제취향이 아니던데 저랑 딱 반대시네요 ㅎㅎ'},
 {'short_name': '모공',
  'desc': '사회주의는 정치체제(반대: 자유주의) 공산주의는 경제체제(반대: 자본주의) 아닌가요...'},
 {'short_name': '모공', 'desc': '아이폰8은...ㅜ'}]